In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# import pandas_profiling
import folium
from folium import plugins

In [2]:
# aisles = pd.read_csv("aisles.csv")
# departments = pd.read_csv("departments.csv")
order_products = pd.read_csv("order_products.csv")
orders = pd.read_csv("orders.csv")
# products = pd.read_csv("products.csv")

In [3]:
user_products = pd.merge(order_products, orders[['order_id','user_id']], how='left', on='order_id')
user_products_1=user_products.drop(['add_to_cart_order'], axis = 1)
user_products_1

,order_id,product_id,user_id
0,2,33120,202279
1,2,28985,202279
2,2,9327,202279
3,2,45918,202279
4,2,30035,202279
...,...,...,...
33819101,3421063,14233,169679
33819102,3421063,35548,169679
33819103,3421070,35951,139822
33819104,3421070,16953,139822


In [4]:
user_products_sample=user_products_1.sample(frac=0.1, random_state=None,axis=0)
user_products_sample.reset_index(drop=True, inplace=True)
user_products_sample

,order_id,product_id,user_id
0,970726,43886,21026
1,1495354,27694,74800
2,1698041,35287,72346
3,808923,45392,90083
4,1555358,28204,180518
...,...,...,...
3381906,933072,15622,60555
3381907,355950,10892,151548
3381908,2792902,24852,30696
3381909,1004962,24852,99904


In [6]:
grouped = user_products_sample.groupby(["user_id","product_id"])["order_id"].min()
grouped_df=pd.DataFrame(grouped)
grouped_df

order_id
user_id product_id          
1       196          2295261
        10258         473747
        12427         473747
        25133         550135
        30450         473747
...                      ...
206209  25837        1889163
        26503        2266710
        36056        2129269
        39216        2977660
        40534        1542354

[2683359 rows x 1 columns]

In [7]:
grouped_df["reordered"]=0

In [11]:
grouped_df.shape

(2683359, 2)

In [9]:
user_products_sample

,order_id,product_id,user_id
0,970726,43886,21026
1,1495354,27694,74800
2,1698041,35287,72346
3,808923,45392,90083
4,1555358,28204,180518
...,...,...,...
3381906,933072,15622,60555
3381907,355950,10892,151548
3381908,2792902,24852,30696
3381909,1004962,24852,99904


In [13]:
user_products_2 = pd.merge(user_products_sample, grouped_df, how='left', on=['order_id','product_id','user_id'])
user_products_2

,order_id,product_id,user_id,reordered
0,970726,43886,21026,0.0
1,1495354,27694,74800,0.0
2,1698041,35287,72346,0.0
3,808923,45392,90083,0.0
4,1555358,28204,180518,0.0
...,...,...,...,...
3381906,933072,15622,60555,0.0
3381907,355950,10892,151548,0.0
3381908,2792902,24852,30696,NaN
3381909,1004962,24852,99904,0.0


In [14]:
user_products_2['reordered'].fillna(1, inplace=True)

In [15]:
user_products_2

,order_id,product_id,user_id,reordered
0,970726,43886,21026,0.0
1,1495354,27694,74800,0.0
2,1698041,35287,72346,0.0
3,808923,45392,90083,0.0
4,1555358,28204,180518,0.0
...,...,...,...,...
3381906,933072,15622,60555,0.0
3381907,355950,10892,151548,0.0
3381908,2792902,24852,30696,1.0
3381909,1004962,24852,99904,0.0


In [16]:
user_products_2[user_products_2["user_id"]==21026]

,order_id,product_id,user_id,reordered
0,970726,43886,21026,0.0
27323,547210,22952,21026,0.0
137436,1139730,5994,21026,0.0
193507,3211368,25745,21026,0.0
781059,1074857,40254,21026,0.0
956563,1259620,4313,21026,0.0
1028379,1402931,10957,21026,1.0
1028754,30312,10957,21026,0.0
1030478,1259620,18212,21026,0.0
1061025,2852274,4313,21026,1.0
